In [ ]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from itertools import combinations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/My Drive/AnnotationDataset_paper/datasets/multi_selection_2.csv')
df

In [ ]:
df_pivot = df.pivot(index='review_id', columns='annotator')
df_pivot.columns = [f'{col[0]}_{col[1]}' for col in df_pivot.columns]

In [ ]:
def calculate_kappa(df, topic):
    annotators = ['evaluator', 'annotator1', 'annotator2']
    kappas = {}
    for i in range(len(annotators)):
        for j in range(i+1, len(annotators)):
            topic_i = f'{topic}_{annotators[i]}'
            topic_j = f'{topic}_{annotators[j]}'

            df_pair = df[[topic_i, topic_j]].dropna()
            kappa = cohen_kappa_score(df_pair[topic_i], df_pair[topic_j])
            kappas[f'{annotators[i]}-{annotators[j]}'] = kappa
    return kappas

In [ ]:
kappa_results = {}
topics = ['sub1_only', 'sub2_only', 'sub3_only']

for topic in topics:
    kappa_results[topic] = calculate_kappa(df_pivot, topic)

average_kappa_results = {}
annotators_pairs = ['evaluator-annotator1', 'evaluator-annotator2', 'annotator1-annotator2']

for pair in annotators_pairs:
    kappa_values = [kappa_results[topic][pair] for topic in topics]
    average_kappa_results[pair] = sum(kappa_values) / len(kappa_values)

for topic, kappas in kappa_results.items():
    print(f'\nKappa scores for {topic}:')
    for annotators, kappa in kappas.items():
        print(f'{annotators}: {kappa:.3f}')

print('\nAverage Kappa scores across all topics:')
for annotators, avg_kappa in average_kappa_results.items():
    print(f'{annotators}: {avg_kappa:.4f}')